In [1]:
from __future__ import print_function
from __future__ import division

import sys
sys.path.insert(0, 'src')
import numpy as np
import transform, vgg, pdb, os
import tensorflow as tf
import cv2
from PIL import Image, ImageFont, ImageDraw

In [2]:
#checkpoint paths 
models=[
{"ckpt":"models/ckpt_cubist_b20_e4_cw05/fns.ckpt", "style":"styles/cubist-landscape-justineivu-geanina.jpg"}, # 1
{"ckpt":"models/ckpt_hokusai_b20_e4_cw15/fns.ckpt", "style":"styles/hokusai.jpg"},							  	# 2
{"ckpt":"models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt", "style":"styles/kandinsky2.jpg"},							# 3
{"ckpt":"models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt", "style":"styles/liechtenstein.jpg"},					# 4
{"ckpt":"models/ckpt_wu_b20_e4_cw15/fns.ckpt", "style":"styles/wu4.jpg"},										# 5
{"ckpt":"models/ckpt_elsalahi_b20_e4_cw05/fns.ckpt", "style":"styles/elsalahi2.jpg"},							# 6
{"ckpt":"models/ckpt_udnie/udnie.ckpt", "style":"styles/udnie.jpg"},											# 7
{"ckpt":"models/ckpt_maps3_b5_e2_cw10_tv1_02/fns.ckpt", "style":"styles/maps3.jpg"},							# 8

{"ckpt":"models/ckpt_kandinsky/fns.ckpt", "style":"styles/kandinsky.jpg"},										# 9
{"ckpt":"models/ckpt_scream/scream.ckpt", "style":"styles/the_scream.jpg"},										# 10
{"ckpt":"models/ckpt_rain_princess/rain_princess.ckpt", "style":"styles/rain_princess.jpg"},					# 11
{"ckpt":"models/ckpt_la_muse/la_muse.ckpt", "style":"styles/la_muse.jpg"},										# 12
{"ckpt":"models/ckpt_wave/wave.ckpt", "style":"styles/wave.jpg"},												# 13
{"ckpt":"models/ckpt_croquis/fns.ckpt", "style":"styles/croquis.png"}											# 14
]

In [3]:
def load_checkpoint(checkpoint, sess):
    saver = tf.train.Saver()
    try:
        saver.restore(sess, checkpoint)
        print('[load_checkpoint] checkpoint: ', checkpoint)
        return True
    except:
        print("checkpoint %s not loaded correctly" % checkpoint)
        return False

In [4]:
def get_img_size(path):
    img = cv2.imread(path)
    return img.shape

In [9]:
def get_img_info(path):
    if os.path.exists(path):
        if os.path.isdir(path):
            fnames = os.listdir(path)
            flist = []
            for img in fnames:
                fpath = os.path.join(source, img)
                flist.append([fpath, img, get_img_size(fpath)])
            return flist
        else:
            img = os.path.split(path)[-1]
            return [path, get_img_size(path)]
    else:
        print("there's no files or directories under the path you provided..")

In [22]:
def main(source_path, *arg):
    idx_model = 0
    device_t='/gpu:0'
    soft_config = tf.ConfigProto(allow_soft_placement=True)
    soft_config.gpu_options.allow_growth = True
    
    #create a folder to save result images if not exists
    save = os.path.join(os.path.dirname(source_path), 'result')
    print("files will be saved under: ", save)
    if not os.path.exists(save):
        os.mkdir(save)
        
    images = get_img_info(source_path)
    
    for i in images:
        g = tf.Graph()
        with g.as_default(), g.device(device_t), tf.Session(config=soft_config) as sess:      
            img_path = i[0]
            img_name = i[1]
            #height = i[2][0]
            height = int(i[2][0]*0.8)
            height = height if height % 4 == 0 else height + 4 - (height % 4) # must be divisible by 4
            #width = i[2][1]
            width = int(i[2][1]*0.8)
            width = width if width % 4 == 0 else width + 4 - (width % 4) # must be divisible by 4
            img_shape = (height, width, 3)
            print(img_shape)
            batch_shape = (1,) + img_shape
            img_placeholder = tf.placeholder(tf.float32, shape=batch_shape, name='img_placeholder')
            preds = transform.net(img_placeholder)

            
            # load checkpoint
            img = cv2.imread(img_path)
            img = cv2.resize(img, (width, height))
            
            #img = cv2.flip(img, 1)
            # 1은 좌우 반전, 0은 상하 반전
            X = np.zeros(batch_shape, dtype=np.float32)
            # print('X: ', X) # X에는 이미지들 있음
            X[0] = img
            
            for idx_model in range(14):
                load_checkpoint(models[idx_model]["ckpt"], sess)
                style = cv2.imread(models[idx_model]["style"])
                output = sess.run(preds, feed_dict={img_placeholder:X})
                output = output[:, :, :, [2,1,0]].reshape(img_shape)
                output = np.clip(output, 0, 255).astype(np.uint8)
                output = cv2.resize(output, (width, height))
                
                name, ext = os.path.splitext(img_name)
                name = name + "_" + str(idx_model) + ext
                # savposco_ceoe image
                save_path = os.path.join(save, name)
                cv2.imwrite(save_path, output)
                

In [23]:
def main2(source_path, *arg):
    fontpath='/usr/share/fonts/truetype/nanum/NanumBarunGothicBold.ttf'
    font= ImageFont.truetype(fontpath, 150)
    idx_model = 0
    device_t='/gpu:0'
    soft_config = tf.ConfigProto(allow_soft_placement=True)
    soft_config.gpu_options.allow_growth = True
    
    #create a folder to save result images if not exists
    save = os.path.join(os.path.dirname(source_path), 'print')
    print("files will be saved under: ", save)
    if not os.path.exists(save):
        os.mkdir(save)
        
    images = get_img_info(source_path)
    
    for i in images:
        g = tf.Graph()
        with g.as_default(), g.device(device_t), tf.Session(config=soft_config) as sess:      
            img_path = i[0]
            img_name = i[1]
            height = i[2][0]
            height = height if height % 4 == 0 else height + 4 - (height % 4) # must be divisible by 4
            width = i[2][1]
            width = width if width % 4 == 0 else width + 4 - (width % 4) # must be divisible by 4
            img_shape = (height, width, 3)
            batch_shape = (1,) + img_shape
            img_placeholder = tf.placeholder(tf.float32, shape=batch_shape, name='img_placeholder')
            preds = transform.net(img_placeholder)

            
            # load checkpoint
            img = cv2.imread(img_path)
            img = cv2.resize(img, (width, height))

            
            #img = cv2.flip(img, 1)
            # 1은 좌우 반전, 0은 상하 반전
            X = np.zeros(batch_shape, dtype=np.float32)
            # print('X: ', X) # X에는 이미지들 있음
            X[0] = img
            
            for idx_model in range(14):
                load_checkpoint(models[idx_model]["ckpt"], sess)
                style = cv2.imread(models[idx_model]["style"])
                output = sess.run(preds, feed_dict={img_placeholder:X})
                output = output[:, :, :, [2,1,0]].reshape(img_shape)
                output = np.clip(output, 0, 255).astype(np.uint8)
                #output = cv2.resize(output, (width, height))
                
                pr_img = np.zeros((3840,5120,3),dtype='uint8')+255
                resizeH = int(3840*1)
                img = cv2.resize(output,(5120, resizeH))
                pr_img[:resizeH,:,:] = img
                img_pil = Image.fromarray(pr_img)
                draw = ImageDraw.Draw(img_pil)
                draw.text((640,3640), '포항공과대학교 인공지능대학원·연구원 방문 기념  2020.09.11', font=font, fill=(255,255,255,0))
                pr_img = np.array(img_pil)
                cv2.imwrite('./print' + '/' 'print.png', pr_img)
                os.system("lpr ./print/print.png")

                

In [25]:
source = "/home/pirl/Desktop/EXCO_CAM/2021_cal" #source image location(source_path could be an absolute path for a folder or a file)
main(source)

files will be saved under:  /home/pirl/Desktop/EXCO_CAM/result
(2764, 4148, 3)
INFO:tensorflow:Restoring parameters from models/ckpt_cubist_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_cubist_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_wu_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_wu_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_elsalahi_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_elsalahi_b20_e4

INFO:tensorflow:Restoring parameters from models/ckpt_croquis/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_croquis/fns.ckpt
(2764, 4148, 3)
INFO:tensorflow:Restoring parameters from models/ckpt_cubist_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_cubist_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_wu_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_wu_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_elsalahi_b20_e4_cw05

[load_checkpoint] checkpoint:  models/ckpt_wave/wave.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_croquis/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_croquis/fns.ckpt
(2764, 4148, 3)
INFO:tensorflow:Restoring parameters from models/ckpt_cubist_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_cubist_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_wu_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_wu_b20_e4_cw15/fns.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from models/ckpt_wave/wave.ckpt
[load_checkpoint] checkpoint:  models/ckpt_wave/wave.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_croquis/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_croquis/fns.ckpt
(2664, 3996, 3)
INFO:tensorflow:Restoring parameters from models/ckpt_cubist_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_cubist_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_wu_b20_e4_cw15/fns.ckpt
[load_checkpoi

[load_checkpoint] checkpoint:  models/ckpt_la_muse/la_muse.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_wave/wave.ckpt
[load_checkpoint] checkpoint:  models/ckpt_wave/wave.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_croquis/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_croquis/fns.ckpt
(2160, 1920, 3)
INFO:tensorflow:Restoring parameters from models/ckpt_cubist_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_cubist_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring pa

INFO:tensorflow:Restoring parameters from models/ckpt_la_muse/la_muse.ckpt
[load_checkpoint] checkpoint:  models/ckpt_la_muse/la_muse.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_wave/wave.ckpt
[load_checkpoint] checkpoint:  models/ckpt_wave/wave.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_croquis/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_croquis/fns.ckpt
(2764, 4148, 3)
INFO:tensorflow:Restoring parameters from models/ckpt_cubist_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_cubist_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_liechtenstein_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  

[load_checkpoint] checkpoint:  models/ckpt_rain_princess/rain_princess.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_la_muse/la_muse.ckpt
[load_checkpoint] checkpoint:  models/ckpt_la_muse/la_muse.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_wave/wave.ckpt
[load_checkpoint] checkpoint:  models/ckpt_wave/wave.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_croquis/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_croquis/fns.ckpt
(2764, 4148, 3)
INFO:tensorflow:Restoring parameters from models/ckpt_cubist_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_cubist_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_hokusai_b20_e4_cw15/fns.ckpt
INFO:tensorflow:Restoring parameters from models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
[load_checkpoint] checkpoint:  models/ckpt_kandinsky_b20_e4_cw05/fns.ckpt
INFO:tensorflow:Restoring parameters from mo